# GomMold Mold Detection – YOLOv8 Training Notebook

**Course**: Software Engineering and AI Application

**Team Members**
- Nur Shaqeerah
- Aina Nur Insyeerah
- Nur Ain Syafiqah
- Yumni Karmila

**Purpose**:  
This notebook trains and evaluates a YOLOv8 model for detecting mold and non-mold areas in household environments.


In [ ]:
#install prerequisites
!pip install ultralytics


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

## 1. Dataset

The dataset was created using Roboflow (mold_detection_GomMold).  
It contains two classes:

- **Mold**
- **No Mold**

The dataset has been augmented with:
- Horizontal flips  
- Brightness/contrast adjustments  
- Exposure variations  

### 🔗 Roboflow Dataset Link
Dataset source (Roboflow project page):

👉 https://universe.roboflow.com/gommold-8opye/mold_detection_gommold-if0p0

This link provides access to dataset details, versions, and annotations.
(download the latest version from there)


In [ ]:

# Download Dataset from Roboflow
!pip install roboflow
from roboflow import Roboflow

# ⚠️ IMPORTANT:
# Insert your own Roboflow API key here if you want to reproduce the experiment.
# The key has been removed for security in the public GitHub version.

rf = Roboflow(api_key="8X3RHgilXHsCh0cPcUSj")
project = rf.workspace("gommold-8opye").project("mold_detection_gommold-if0p0")
version = project.version(4)
dataset = version.download("yolov8")




In [ ]:
#if u downloaded the data manually from the link, unzip the file

!unzip /content/mold_detection_GomMold.yolov8.zip -d /content/


In [ ]:
!ls "/content/gdrive/MyDrive/GomMold_AIML"


## 2. Load YOLOv8 Model
We start training from `yolov8n.pt` using transfer learning.

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

## 3. Train the Model


In [ ]:
model.train(
    data="/content/mold_detection_GomMold-4/data.yaml",  # path to dataset
    epochs=50,        # 30–50 recommended
    imgsz=640,        # image size
    batch=16,
    patience=10,      # early stopping if no improvement
    lr0=0.01,         # initial learning rate
    augment=True      # enable augmentation
)

In [ ]:
# Mount the trained model to drive

!cp /content/runs/detect/train/weights/best.pt /content/gdrive/MyDrive/GomMold_AIML/new_best4.pt #change this to your drive directory


## 4. Training Metrics & Graphs
YOLO automatically generates:
- results.png
- loss curves
- mAP curves
- confusion matrix


In [ ]:
from IPython.display import Image

Image('/content/runs/detect/train/results.png')


In [ ]:
Image('/content/runs/detect/train/confusion_matrix.png')

In [ ]:
!pip install firebase-admin


In [ ]:
from ultralytics import YOLO

model = YOLO("/content/runs/detect/train/weights/best.pt")

model.export(format="onnx", opset=12)


## 5. Inference Samples


In [ ]:
!mkdir sample_test

# copy a few images from test set
!cp /content/mold_detection_GomMold-4/test/images/*.jpg sample_test/

# run prediction
results = model.predict("/content/sample_test", save=True)


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import firebase_admin

# reset if already initialized
if firebase_admin._apps:
    firebase_admin.delete_app(firebase_admin.get_app())


In [ ]:
cred = credentials.Certificate("/content/gommold-c6654-firebase-adminsdk-fbsvc-bf7bea4cc9.json")

firebase_admin.initialize_app(cred, {
    "storageBucket": "gommold-c6654.firebasestorage.app"
})


In [ ]:
bucket = storage.bucket()


In [ ]:
blob = bucket.blob("models/best.onnx")
blob.upload_from_filename("/content/runs/detect/train/weights/best.onnx")
blob.make_public()

print("MODEL URL:", blob.public_url)


## 8. Conclusion

The YOLOv8 model performed well in detecting mold vs non-mold images.  
Further improvements include:
- Increasing dataset variety
- Improving low-light images
- Adding more “No Mold” clean wall images to reduce false positives


In [ ]:
#test onnx file tadi

In [ ]:
!pip install onnxruntime


In [ ]:
import onnxruntime as ort
import numpy as np
from PIL import Image
import cv2


In [ ]:
onnx_path = "/content/runs/detect/train/weights/best.onnx"
session = ort.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])

print("ONNX loaded successfully!")


In [ ]:
from google.colab import files
uploaded = files.upload()



In [ ]:
img = cv2.imread("/content/1-b2ad256f.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = cv2.resize(img, (640, 640))

# normalize
input_data = img_resized / 255.0
input_data = np.transpose(input_data, (2, 0, 1))  # HWC → CHW
input_data = np.expand_dims(input_data, axis=0).astype(np.float32)


In [ ]:
outputs = session.run(None, {"images": input_data})
print("ONNX inference ran successfully!")
print("Output length:", len(outputs))


In [ ]:
outputs = session.run(None, {"images": input_data})
print("ONNX inference ran successfully!")
print("Output length:", len(outputs))

output = outputs[0]
print("Output shape:", output.shape)


In [ ]:
!pip install onnxruntime opencv-python pillow numpy



In [ ]:
import onnxruntime as ort
import numpy as np
import cv2

onnx_path = "/content/runs/detect/train/weights/best.onnx"
session = ort.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])


In [ ]:
from google.colab import files
uploaded = files.upload()

test_image_path = list(uploaded.keys())[0]  # takes the uploaded image


In [ ]:
img = cv2.imread(test_image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = cv2.resize(img_rgb, (640, 640))

input_data = img_resized / 255.0
input_data = np.transpose(input_data, (2, 0, 1))  # HWC → CHW
input_data = input_data[np.newaxis, :, :, :].astype(np.float32)  # add batch dim


In [ ]:
outputs = session.run(None, {"images": input_data})
pred = outputs[0]   # shape (1, 6, 8400)


In [ ]:
pred = pred[0]  # remove batch dim

# extract each component
x = pred[0]
y = pred[1]
w = pred[2]
h = pred[3]
conf = pred[4]
cls = pred[5]


In [ ]:
score_threshold = 0.25  # adjust if needed
mask = conf > score_threshold

x = x[mask]
y = y[mask]
w = w[mask]
h = h[mask]
conf = conf[mask]
cls = cls[mask]


In [ ]:
boxes = []

for i in range(len(x)):
    xc, yc, bw, bh = x[i], y[i], w[i], h[i]
    xmin = int((xc - bw/2) * img.shape[1] / 640)
    ymin = int((yc - bh/2) * img.shape[0] / 640)
    xmax = int((xc + bw/2) * img.shape[1] / 640)
    ymax = int((yc + bh/2) * img.shape[0] / 640)
    boxes.append([xmin, ymin, xmax, ymax, conf[i]])


In [ ]:
for (xmin, ymin, xmax, ymax, score) in boxes:
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
    cv2.putText(img, f"mold: {score:.2f}", (xmin, ymin - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(img)


In [ ]:
print("Detections:", len(boxes))
for b in boxes:
    print("Box:", b)


In [ ]:
import onnxruntime as ort
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# -------------------------
# 1. Load ONNX model
# -------------------------
onnx_path = "/content/models_best.onnx"
session = ort.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])

# -------------------------
# 2. Upload test image
# -------------------------
from google.colab import files
uploaded = files.upload()
test_image_path = list(uploaded.keys())[0]

# -------------------------
# 3. Preprocess image
# -------------------------
img = cv2.imread(test_image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h0, w0 = img.shape[:2]  # original dims

img_resized = cv2.resize(img_rgb, (640, 640))
input_data = img_resized / 255.0
input_data = np.transpose(input_data, (2, 0, 1))  # HWC → CHW
input_data = np.expand_dims(input_data, axis=0).astype(np.float32)

# -------------------------
# 4. Inference
# -------------------------
outputs = session.run(None, {"images": input_data})
pred = outputs[0][0]   # shape -> (6, 8400)

# -------------------------
# 5. Extract components
# -------------------------
x = pred[0]
y = pred[1]
w = pred[2]
h = pred[3]
conf = pred[4]
cls = pred[5]

# -------------------------
# 6. Filter by confidence
# -------------------------
threshold = 0.30
mask = conf > threshold

x = x[mask]
y = y[mask]
w = w[mask]
h = h[mask]
conf = conf[mask]

# -------------------------
# 7. Convert XYWH → XYXY (scaled back to original img)
# -------------------------
boxes = []
for i in range(len(x)):
    xc, yc, bw, bh = x[i], y[i], w[i], h[i]
    xmin = int((xc - bw/2) * w0 / 640)
    ymin = int((yc - bh/2) * h0 / 640)
    xmax = int((xc + bw/2) * w0 / 640)
    ymax = int((yc + bh/2) * h0 / 640)
    boxes.append([xmin, ymin, xmax, ymax, conf[i]])

# -------------------------
# 8. Print detections
# -------------------------
print("Detections:", len(boxes))
for b in boxes:
    print("Box:", b)

# -------------------------
# 9. Draw boxes
# -------------------------
for (xmin, ymin, xmax, ymax, score) in boxes:
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
    cv2.putText(img, f"Mold {score:.2f}", (xmin, ymin - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

# -----------------


In [ ]:
from google.colab.patches import cv2_imshow


In [ ]:
cv2_imshow(img)


In [ ]:
# DRAW FINAL BOXES
for (xmin, ymin, xmax, ymax, score) in boxes:
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0,255,0), 2)
    cv2.putText(img, f"Mold {score:.2f}", (xmin, ymin-5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

# SHOW IMAGE
from google.colab.patches import cv2_imshow
cv2_imshow(img)


In [ ]:
print(session.get_outputs())


In [ ]:
out = session.get_outputs()[0]
print("Output name:", out.name)
print("Output shape:", out.shape)
print("Output type:", out.type)
